<a href="https://colab.research.google.com/github/lukkychan/scaling_to_high/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir original_video
!mkdir org_no_sound
!mkdir -p frames/1080p
!mkdir -p frames/360p
!mkdir scaled

Downloading the videos

In [ ]:
!wget "https://rr3---sn-i5h7lnll.googlevideo.com/videoplayback?expire=1687873118&ei=_pGaZK_yFZuWx_AP9c2bgAk&ip=194.163.187.202&id=o-ABQoFFKI-0TYpItnbUBExAW_WFhBuG8nGHo1l4CFhAvB&itag=18&source=youtube&requiressl=yes&mh=5x&mm=31%2C26&mn=sn-i5h7lnll%2Csn-5hne6n6l&ms=au%2Conr&mv=m&mvi=3&pl=24&initcwndbps=591250&spc=qEK7BzQe1PdnWmhw8X7fm6G_bDBY6vk&vprv=1&svpuc=1&mime=video%2Fmp4&gir=yes&clen=7096879&ratebypass=yes&dur=124.435&lmt=1687436485346821&mt=1687851193&fvip=5&fexp=24007246&c=ANDROID&txp=6319224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRAIgLi21odpgQulN_4_GdFEVFpyniuUl6h2NNK50TfYENhgCIBr5HRx5w7eChj4_mdEOZuKebqN-POQwmCYhW85TJNWc&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRgIhAJi6bx2nbfyG0lLVEz5H7xOzg6wBfbf88AGS1mgfisRgAiEA8vRCYhGDORRcsZbcL4glRFkSgTxTBoVgbHCzDq2qNR0%3D&title=SPY%20Trailer%20(Telugu)%20|%20Nikhil%20Siddharth%20|%20Garry%20BH%20|%20Charantej%20Uppalapati%20|%20Ishwarya%20Menon"

Removing the audios

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

def remove_audio(input_video_path, output_video_path):
    # Load the video clip
    video = VideoFileClip(input_video_path)

    # Remove the audio
    video_without_audio = video.without_audio()

    # Save the video without audio
    video_without_audio.write_videofile(output_video_path, codec='libx264')

    # Close the video clip
    video.close()

# Specify the paths to the input video and desired output video
input_video_path = '/content/original_video/360p.mp4'
output_video_path = '/content/org_no_sound/360p.mp4'

# Call the function to remove audio
remove_audio(input_video_path, output_video_path)
print("done.")


Creating Frames

In [ ]:
import cv2

# Specify the path to the video file
video_path = "/content/org_no_sound/360p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/frames/360p/"

# Specify the maximum number of frames to extract
max_frames = 150

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames // max_frames, 1)

# Initialize a counter to keep track of the extracted frames
frame_count = 0

# Loop through the frames and extract the desired number of frames
while frame_count < max_frames:
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}{frame_count:03d}.jpg"
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

# Release the video capture object
video.release()
print("done.")

Scaling the images to 1080p

In [ ]:
import os
import subprocess

def scale_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    for image_file in image_files:
        # Construct the input and output file paths
        input_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, image_file)

        # Execute FFmpeg command to scale the image to 1920x1080
        command = ['ffmpeg', '-i', input_path, '-vf', 'scale=1920:1080', output_path]
        subprocess.call(command)

# Specify the paths to the input folder and output folder
input_folder = '/content/frames/360p/'
output_folder = '/content/scaled/'

# Call the function to scale the images
scale_images(input_folder, output_folder)
print("scaling done.")


model training

In [ ]:
import os
import numpy as np
import cv2
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

# Step 1: Organize data and directories
original_dir = '/content/frames/1080p/'  # Directory containing original 1080p images
scaled_dir = '/content/scaled/'  # Directory containing scaled images

# Step 2: Load and preprocess the images
original_images = []
scaled_images = []

for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        original_path = os.path.join(original_dir, filename)
        scaled_path = os.path.join(scaled_dir, filename)

        original_image = cv2.imread(original_path)
        scaled_image = cv2.imread(scaled_path)

        # Preprocess the images (resize and normalize)
        original_image = cv2.resize(original_image, (256, 256)) / 255.0
        scaled_image = cv2.resize(scaled_image, (256, 256)) / 255.0

        original_images.append(original_image)
        scaled_images.append(scaled_image)

original_images = np.array(original_images)
scaled_images = np.array(scaled_images)

# Step 3: Define the U-Net model architecture
inputs = Input(shape=(256, 256, 3))
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
up5 = UpSampling2D(size=(2, 2))(conv4)

up5 = Conv2D(256, 2, activation='relu', padding='same')(up5)
merge5 = concatenate([conv3, up5], axis=3)
conv5 = Conv2D(256, 3, activation='relu', padding='same')(merge5)
conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
up6 = UpSampling2D(size=(2, 2))(conv5)

up6 = Conv2D(128, 2, activation='relu', padding='same')(up6)
merge6 = concatenate([conv2, up6], axis=3)
conv6 = Conv2D(128, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)
up7 = UpSampling2D(size=(2, 2))(conv6)

up7 = Conv2D(64, 2, activation='relu', padding='same')(up7)
merge7 = concatenate([conv1, up7], axis=3)
conv7 = Conv2D(64, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

output = Conv2D(3, 1, activation='sigmoid')(conv7)

model = Model(inputs=inputs, outputs=output)

# Step 4: Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(scaled_images, original_images, batch_size=32, epochs=10, validation_split=0.2)

# Step 5: Save the trained model
model.save('/content/model.h5')

# Step 6: Evaluate the model (optional)
# test_images = [...]  # Load test images
# predicted_images = model.predict(test_images)

# Step 7: Inference on new scaled images (optional)
# new_scaled_image = cv2.imread('path/to/new/scaled/image.jpg')
# new_scaled_image = cv2.resize(new_scaled_image, (256, 256)) / 255.0
# predicted_image = model.predict(np.array([new_scaled_image]))
# predicted_image = predicted_image[0] * 255.0
# cv2.imwrite('path/to/save/predicted/image.jpg', predicted_image)


Epoch 1/10
4/4 [==============================] - 730s 180s/step - loss: 0.1742 - val_loss: 0.1662
Epoch 2/10
4/4 [==============================] - 751s 191s/step - loss: 0.1138 - val_loss: 0.0651
Epoch 3/10
4/4 [==============================] - 719s 180s/step - loss: 0.0542 - val_loss: 0.0651
Epoch 4/10
3/4 [=====================>........] - ETA: 2:57 - loss: 0.0548